# Point sources mfds

In [ ]:
%matplotlib inline
import re
import os
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from matplotlib.path import Path
from matplotlib.collections import PathCollection

sys.path.append('/Users/mpagani/Repos/original/oq-engine/')
sys.path.append('/Users/mpagani/Repos/oq-man/')

import oqman.man.checking_utils.source_model_utils as model
from oqman.man.single_source_utils.tools import get_discrete_mfds

## Model to be analysed

In [ ]:
path = '/Users/mpagani/Prj/2017/global_modelling/results/pkl/*.pkl'
modell = set(['arm17'])
modell = set(['am17a'])
modell = set(['am17b'])
modell = set(['am17c'])
modell = set(['am17d'])

## Read data for the different models

In [ ]:
modd = {}
for fname in glob.glob(path):
    slist = re.split('\.',  os.path.basename(fname)) 
    if re.search('[a-z]*[0-9]*\.pkl', fname) and slist[0] in modell:
        print (fname, end=' ')
        mod, points, info, sidx = model.load(fname) 
        print ('done')
        modd[slist[0]] = {'model': mod, 'info': info, 'points': points}
print (modd.keys())

## Plotting

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1)

mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12

for key in sorted(modd.keys()):
    print ('key: ',key)   
    print ('')

    ids = [s.source_id for s in modd[key]['points']]
    mfds = get_discrete_mfds(modd[key]['points'])

    paths = []
    mmax = []
    for mfd in mfds:
        paths.append(Path(mfd.squeeze()))
        mmax.append(max(mfd[:,0]))

    cols = np.random.rand(len(mfds), 3)
    paths_coll = PathCollection(paths, edgecolors=cols)
    ax.add_collection(paths_coll)
    plt.grid(which='both')
    ax.autoscale()
    plt.yscale('log')
    plt.xlabel('Magnitude, m', fontsize=14)
    plt.ylabel('Annual rate of occurrence, m', fontsize=14)
    plt.title(key)
    plt.ylim([1e-13, 1e-3])

In [ ]:
fig=plt.figure(figsize=(8,6))
bins = np.arange(5, 8, step=0.1)
print (len(bins))

his, _ = np.histogram(mmax, bins)
imin = np.min(np.nonzero(his>0))
imax = np.max(np.nonzero(his>0))

# plotting histogram
dlt = 1
plt.bar(bins[imin:imax+dlt], 
        his[imin:imax+dlt]/float(sum(his[imin:imax+dlt])), 
        width=0.1,
        edgecolor='black',
        align='edge')
plt.xlabel('Magnitude, m', fontsize=12)
plt.grid(which='both')
plt.title(key)
plt.xlim([5.9, 7.5])
plt.ylim([0.0, 1.0])